This notebook is intended to demonstrate how vessel segmentation methods of ITKTubeTK can be applied to segment vessels passing through specific voxels.

In [2]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [3]:
ImageType = itk.Image[itk.F, 3]

im1vess = itk.imread("../Data/MRI-Cases/mra.mha",itk.F)

In [5]:
res = ttk.ResampleImage[ImageType].New()
res.SetInput(im1vess)
res.SetMakeHighResIso(True)
res.Update()
im1iso = res.GetOutput()

In [6]:
xp1=[-3.9,-54.8,12.3]
xp2=[18.4,-16,-21]
xp3=[-46,-15,-13]

vSeg = ttk.SegmentTubes[ImageType].New()
vSeg.SetInput(im1iso)
vSeg.SetVerbose(False)
vSeg.SetRadiusInObjectSpace( 0.33 )
vSeg.SetMinRoundness(0.2)
vSeg.SetVerbose(True)
tube = vSeg.ExtractTubeInObjectSpace( xp1, 1 )
tube = vSeg.ExtractTubeInObjectSpace( xp2, 2 )
tube = vSeg.ExtractTubeInObjectSpace( xp3, 3 )
tubeG = vSeg.GetTubeGroup()

In [7]:
# Example of accessing data in a tube
p = list(map(lambda x: tube.GetPoint(x).GetPositionInObjectSpace(), range(tube.GetNumberOfPoints())))
r = list(map(lambda x: tube.GetPoint(x).GetNormal1InObjectSpace()[0], range(tube.GetNumberOfPoints())))
print(r)

[0.9476970880991076, 0.9495690175696083, 0.9266045644116354, 0.9367415567502634, 0.9155456729334054, 0.9213089192308838, 0.05176911784905983, 0.9391687288692704, 0.9384839625170907, 0.9606947909606186, 0.9582234405369334, 0.9611833306558912, 0.36401733678689396, 0.37340907325503025, 0.38483192749684514, 0.39505296606961043, 0.404485075871499, 0.4214139959928322, 0.5813682044136994, 0.9425720597055746, 0.9388275823783475, 0.837780040382554, 0.5486228132292076, 0.5954474236158476, 0.5834204618487692, 0.6080738986393774, 0.874378739619728, 0.9311981977165955, 0.7575598765394175, 0.9060316618735894, 0.6978052949619998, 0.8608796275716065, 0.8223879381650364, 0.8394487738942488, 0.22139601719854918, 0.07743398044471152, -0.024163553441872127, -0.16960091654091336, -0.3546832216099779, -0.2399673675262435, -0.380246384781714, -0.3796410194269799, -0.902100574860639, -0.8030052934423455, 0.17043773833620232, 0.7936997414336965, -0.4308000859686011, 0.46682757964753885, 0.5141901074183847, 0.5

In [9]:
TubesToImageFilterType = ttk.ConvertTubesToImage[ImageType]
tubesToImageFilter = TubesToImageFilterType.New()
tubesToImageFilter.SetUseRadius(True)
tubesToImageFilter.SetColorByRidgeness(True)
tubesToImageFilter.SetTemplateImage(im1vess)
tubesToImageFilter.SetInput(tubeG)
tubesToImageFilter.Update()

outputImage = tubesToImageFilter.GetOutput()
view(outputImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [10]:
writer = itk.SpatialObjectWriter[3].New()
writer.SetFileName("vessels.tre")
writer.SetInput(tubeG)
writer.Update()